In [1]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))


In [2]:
res_all <- readRDS( 'finemap_results_figure3.rds' )


In [3]:
## Define six colors
colors6 <- c(
	rgb( 107, 223, 72,  max = 255 ),
	rgb( 121, 213, 117, max = 255 ),
	rgb( 129, 214, 161, max = 255 ),
	rgb( 93,  217, 206, max = 255 ),
	rgb( 168, 226, 242, max = 255 ),
	rgb( 101, 193, 227, max = 255 )
)


In [4]:
cascade_files <- file.path('../cascade', 'cascade.input.files.tsv')
traits <- fread(cascade_files)
dim(traits) %>% print()


[1] 35  4


In [5]:
length(intersect(names(res_all), (traits %>% select(annotation) %>% pull())))


[1] 35

In [6]:
res <- list()
for(trait in intersect(names(res_all), (traits %>% select(annotation) %>% pull()))){
    res[[ trait ]] <- res_all[[ trait ]]
}


In [7]:
compute_finemap_sumstats <- function(trait, res_trait){
    n_regions           <- nrow( drop_na ( res_trait ) )
    n_signals           <- sum( as.numeric( drop_na( res_trait )$n_signals ) ) 
    n_signal_per_region <- table(
                cut(
                    x      = as.numeric( drop_na( res_trait )$n_signals ),
                    breaks = c( 0, 1, 2, 3, 4, 5, Inf ),
                    labels = c( 'sr1', 'sr2', 'sr3', 'sr4', 'sr5', 'sr6+' )
                )
        )
    n_snps_per_signal   <- table(
                cut( 
                    x      = unlist( sapply( drop_na( res_trait )$n_snps_per_signal, function( x ) as.numeric( strsplit( x, split = '\\|' )[[ 1 ]] ), USE.NAMES = F ) ),
                    breaks = c( 0, 1, 5, 10, 20, 50, Inf ),
                    labels = c( 'ss1', 'ss2-5', 'ss6-10', 'ss11-20', 'ss21-50', 'ss51+' )
                )
    )
                    
    finemap_sumstats <- c(trait, n_regions, n_signals, n_signal_per_region, n_snps_per_signal)
    names(finemap_sumstats)[1:3] <- c('trait', 'n_regions', 'n_signals')

    finemap_sumstats
}

In [8]:
count_df <- traits %>% select(name, annotation) %>% 
rename('trait' = 'annotation') %>%
right_join(
    do.call(rbind, lapply(
        intersect(names(res_all), (traits %>% select(annotation) %>% pull())),
        function(trait) { compute_finemap_sumstats(trait, res[[ trait ]]) }
    )) %>% as.data.frame(stringsAsFactors = F),
    by='trait'
)


In [9]:
count_df %>%
fwrite('plotFinemapFigure3_count_summary.tsv', sep='\t')